## Sentiment Experiment Runner Template
10/23/16 - Basic pipeline to run sentiment experiments on the IMDB movie review corpus.
Uses the IMDB dataset folder (http://ai.stanford.edu/~amaas/data/sentiment/).

In [1]:
import os
import math
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
imdb_folder_location = "../aclImdb" # Change this to wherever your imbd folder is located

/Users/Cathy/anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
def imdb_sentiment_reader(dataset_type='train', sentiment='both', folder_location=imdb_folder_location):
    """
    Iterator over the imdb dataset.
    Args:
        is_train: ['train', 'val', 'test] - whether to iterate over the train, val, or test sets.
        sentiment: ['pos', 'neg', 'both']: whether to iterate over just the positive, just the
                   negative, or both.
    Returns: Iterator over (filename, movie_review, sentiment_score) tuples. 
    """
    subfolder = 'train' if dataset_type=='train' else 'test'
    dataset_path = os.path.join(folder_location, subfolder)
    if sentiment=='pos' or sentiment=='both':
        # Sort by the index
        filenames = sorted(os.listdir(os.path.join(dataset_path, 'pos')), 
                                key=lambda filename: int(filename.split('_')[0]))
        # Take a slice if these are for val/test
        if dataset_type == 'val' or dataset_type == 'test':
            cutoff = int(math.ceil(len(filenames) * .2))
            if dataset_type == 'val':
                filenames = filenames[:cutoff]
            else:
                filenames = filenames[cutoff:]
        for filename in filenames:
            sentiment_score = int(filename.split('_')[1].split('.')[0])
            with open(os.path.join(dataset_path, 'pos', filename)) as f:
                review = f.read()
            yield filename, review.decode('utf-8'), sentiment_score
    if sentiment=='neg' or sentiment=='both':
        # Sort by the index 
        filenames = sorted(os.listdir(os.path.join(dataset_path, 'neg')), 
                                key=lambda filename: int(filename.split('_')[0]))
         # Take a slice if these are for val/test
        if dataset_type == 'val' or dataset_type == 'test':
            cutoff = int(math.ceil(len(filenames) * .2))
            if dataset_type == 'val':
                filenames = filenames[:cutoff]
            else:
                filenames = filenames[cutoff:]
        for filename in filenames:
            sentiment_score = int(filename.split('_')[1].split('.')[0])
            with open(os.path.join(dataset_path, 'neg', filename)) as f:
                review = f.read()
            yield filename, review.decode('utf-8'), sentiment_score
    
# ExampleUsage
for filename, review, score in imdb_sentiment_reader(dataset_type='test', sentiment='both'):
    print filename, review, score
    break

2500_8.txt This movie was a sicky sweet cutesy romantic comedy, just the kind of movie I usually dislike but this one was just cute enough to keep me interested. It was really funny in one moment (probably why I liked it) and then just as serious in the next. Plus, it had Ellen in it and I've always had a soft spot for her.<br /><br />Basically, the owner of a book store, Helen (Kate Capshaw) finds a love letter in one of the old couches in her store. She thinks it is for her and goes crazy trying to figure out who sent it. She has kind of shut herself off from the world, so it really throws her for a loop. Eventually, almost everyone connected with her finds this letter and they are all getting mixed signals which creates some really funny moments.<br /><br />Like I said, I am usually not one for this type of movie but I really wound up enjoying it and recommend it highly. 8


In [99]:
class DefaultEvaluator():
    """
    Default evaluator for the sentiment IMDB problem.
    Uses the default evaluation metric defined in the paper.
    """
    def __init__(self, verbose=False):
        self.verbose=verbose
        self.sentiment_analyzer = SentimentIntensityAnalyzer()
        self.sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.build_bigram_lists()
    
    def get_bigrams(self, tokens):
        # Gets bigrams from lowercased tokens
        return [(tokens[i].lower(), tokens[i+1].lower()) for i in range(len(tokens) - 1)]
    
    def build_bigram_lists(self):
        """
        Build lists of all positive and negative training bigrams.
        """
        self.pos_bigrams = set()
        print "Building positive bigram list..."
        for index, (filename, review, score) in enumerate(
            imdb_sentiment_reader(dataset_type='train', sentiment='pos')):
            try:
                tokens = word_tokenize(review)
                self.pos_bigrams.update(self.get_bigrams(tokens))
            except:
                print "Failed to tokenize: pos " + filename
            if self.verbose and index % 1000 == 0:
                print "Now on: " + str(index)
    
        print "Building negative bigram list..."     
        self.neg_bigrams = set()
        for index, (filename, review, score) in enumerate(
            imdb_sentiment_reader(dataset_type='train', sentiment='neg')):
            try:
                tokens = word_tokenize(review)
                self.neg_bigrams.update(self.get_bigrams(tokens))
            except:
                print "Failed to tokenize: neg " + filename
            if self.verbose and index % 1000 == 0:
                print "Now on: " + str(index)
                
    def _convert_imdb_sent_score(self, imdb_score):
        """
        Converts the IMDB sentiment score (on a 0-10 scale) to a binary (-1, 1) sentiment score.
        """
        # IMDB metric: positive is >=7, negative is <= 4
        if imdb_score >= 7:
            return 1.0
        elif imdb_score <= 4:
            return -1.0
        else:
            raise Exception('IMDB score not >=7 or <= 4')
    
    def _convert_vader_sent_score(self, vader_score):
        """
        Converts the vader score to its valence (negative scores are negative, positive are positive).
        """
        return -1.0 if vader_score < 0 else 1.0
    
    def _get_sentiment_changed_score(self, old_score, new_review):
        """
        Return 1 if sentiment changed, -1 if not. Uses the NLTK Vader sentiment analyzer.
        """
        sentiment_score = self._convert_vader_sent_score(
            self.sentiment_analyzer.polarity_scores(new_review)['compound'])
        old_sent_score = self._convert_imdb_sent_score(old_score)
        return 1.0 if (sentiment_score != old_sent_score) else -1.0
    
    def _get_sentence_score(self, old_review, new_review):
        """
        Return a closeness score based on the similarity of number of sentences.
        """
        num_old_sentences = len(self.sentence_tokenizer.tokenize(old_review))
        num_new_sentences = len(self.sentence_tokenizer.tokenize(new_review))
        return min(num_old_sentences/float(num_new_sentences), num_new_sentences/float(num_old_sentences))
    
    def _get_proper_noun_score(self, tagged_old_review, tagged_new_review):
        """
        Returns a metric between 0-1 based on the overlap between proper nouns in the two reviews.
        """
        proper_nouns_old = set([token for token in tagged_old_review if token[1] == 'NNP'])
        proper_nouns_new = set([token for token in tagged_new_review if token[1] == 'NNP'])
        num_overlap = len(proper_nouns_old.intersection(proper_nouns_new))
        num_nouns = max(len(proper_nouns_old), len(proper_nouns_new))
        if num_nouns == 0:
            return 0
        else:
            return float(num_overlap) / max(len(proper_nouns_old), len(proper_nouns_new))
    
    def _get_pos_score(self, tagged_old_review, tagged_new_review):
        """
        Returns a value between 0 and 1 based on the multiset overlap between the POS tags in 
        both reviews.
        """
        pos_tags_old=[tagged_token[1] for tagged_token in tagged_old_review]
        pos_tags_new=[tagged_token[1] for tagged_token in tagged_new_review]
        intersection = Counter(pos_tags_old) & Counter(pos_tags_new)
        num_overlap = sum(intersection.values())
        num_pos = max(len(pos_tags_old), len(pos_tags_new))
        if num_pos == 0:
            return 0
        else:
            return float(num_overlap) / num_pos
    
    def _get_closeness_score(self, old_review, new_review, tokenized_old_review, tokenized_new_review):
        """
        Return a value between 0 and 1 evaluating how similar the old_review is in lingustic 
        structure to the new one.
        Calculated as (proper noun overlap) * (# sentences score) * (# POS score)
        """
        tagged_old_review = nltk.pos_tag(tokenized_old_review)
        tagged_new_review = nltk.pos_tag(tokenized_new_review)
        
        sentence_score = self._get_sentence_score(old_review, new_review)
        proper_noun_score = self._get_proper_noun_score(tagged_old_review, tagged_new_review)
        pos_score = self._get_pos_score(tagged_old_review, tagged_new_review)
        return sentence_score * proper_noun_score * pos_score
    
    def _get_typical_language_score(self, tokenized_old_review, old_score, tokenized_new_review):
        """
        Returns a value between 0 and 1 evaluating how similar the new_review is to existing reviews of 
        its desired sentiment.
        Concretely, evaluates the fraction of bigrams in the new_review that appear in either 
        all other training data reviews of the same sentiment, or within the old_review.
        """
        # Note: make sure to compare to other reviews that are the opposite of the old_score,
        # NOT the vader score.
        old_review_bigrams = set(self.get_bigrams(tokenized_old_review))
        new_review_bigrams = self.get_bigrams(tokenized_new_review)
        # Target sentiment is opposite of the old review sentiment
        target_sentiment = -1 * self._convert_imdb_sent_score(old_score) 
        num_similar_bigrams = 0
        for bigram in new_review_bigrams:
            if (bigram in old_review_bigrams) or \
            (target_sentiment > 0 and bigram in self.pos_bigrams) or \
            (target_sentiment < 0 and bigram in self.neg_bigrams):
                num_similar_bigrams += 1
        if len(new_review_bigrams) == 0:
            return 0
        else:
            return num_similar_bigrams / float(len(new_review_bigrams))
        
    def evaluate(self, filename, old_review, new_review, old_score):
        """
        Evaluation based on a sentiment changed score, closeness score, and 'typical language' score.
        """
        tokenized_old = word_tokenize(old_review)
        tokenized_new = word_tokenize(new_review)
        sent_change = self._get_sentiment_changed_score(old_score, new_review)
        closeness = self._get_closeness_score(old_review, new_review, tokenized_old, tokenized_new)
        typicality = self._get_typical_language_score(tokenized_old, old_score, tokenized_new)
        return sent_change * closeness * typicality

# Example Usage - uses the baseline transformer defined below
"""
evaluator = DefaultEvaluator(verbose=True)
for (filename, review, score) in imdb_sentiment_reader(dataset_type='val', sentiment='pos'): 
    transformed = baseline_transform_func(filename, review, score)
    print review
    print transformed
    print "Evaluation score: " + str(evaluator.evaluate(filename, review, transformed, score))
    break
"""

'\nevaluator = DefaultEvaluator(verbose=True)\nfor (filename, review, score) in imdb_sentiment_reader(dataset_type=\'val\', sentiment=\'pos\'): \n    transformed = baseline_transform_func(filename, review, score)\n    print review\n    print transformed\n    print "Evaluation score: " + str(evaluator.evaluate(filename, review, transformed, score))\n    break\n'

In [100]:
class ExperimentRunner():
    """
    Runs a sentiment experiment runner experiment. 
    Trains on the training set, then iterates over the reviews in the test set,
    transforming them using the transform_func and evaluating them using the eval_func.
    
    Outputs the average performance on the test set.
    
    Args:
        train_reader: an iterator over (filename, review, score) tuples.
        test_reader: an iterator over (filename, review, score) tuples.
        transform_func: should take (filename, review, score) and return a transformed string review.
        eval_func: should take (filename, old_review, new_review, old_sentiment_score) and return a score.
        verbose: default = False
    """
    def __init__(self, train_reader, test_reader, transform_func, evaluator=None, verbose=False):
        self.train_reader = train_reader
        self.test_reader = test_reader
        self.transform_func = transform_func
        if evaluator is None:
            self.evaluator = DefaultEvaluator()
            self.eval_func = self.evaluator.evaluate
        else:
            self.eval_func = evaluator.evaluate
        self.verbose = verbose
        self.scores = []
    
    def run_experiment(self):
        # Iterate over the test set, transforming the reviews and evaluating them
        for index, (filename, review, sent_score) in enumerate(self.test_reader):
            if self.verbose and index % 500 == 0:
                print "Now evaluating: " + str(index)
                # Print the running mean
                print "Current mean score: " + str(np.mean(self.scores))
            # Transform the review
            transformed_review = self.transform_func(filename, review, sent_score)
            # Evaluate the transformed review
            new_score = self.eval_func(filename, review, transformed_review, sent_score)
            self.scores.append(new_score)
        if self.verbose:
            print "Finished evaluating " + str(index) + " test reviews."
        print "Mean score: " + str(np.mean(self.scores))
    
# Example usage
demo_train = imdb_sentiment_reader(dataset_type='train', sentiment='both')
demo_test= imdb_sentiment_reader(dataset_type='val', sentiment='both')
"""
def demo_transform_func(filename, review, score):
    return review

class DemoEval:
    def __init__(self):
        pass
    def evaluate(self, filename, old_review, new_review, old_score):
        return old_score

demo_runner = ExperimentRunner(demo_train, demo_test, demo_transform_func, 
                               evaluator=DemoEval(), verbose=True)
demo_runner.run_experiment()
"""

'\ndef demo_transform_func(filename, review, score):\n    return review\n\nclass DemoEval:\n    def __init__(self):\n        pass\n    def evaluate(self, filename, old_review, new_review, old_score):\n        return old_score\n\ndemo_runner = ExperimentRunner(demo_train, demo_test, demo_transform_func, \n                               evaluator=DemoEval(), verbose=True)\ndemo_runner.run_experiment()\n'

## Baseline

In [10]:
import nltk
from nltk import word_tokenize

In [26]:
def baseline_transform_func(filename, review, score):
    """
    Baseline: returns a review with 'not' inserted in front of any identified adjectives/adverbs.
    """
    tagged_review = nltk.pos_tag(word_tokenize(review))
    transformed_review = []
    for tagged_word in tagged_review:
        if tagged_word[1] in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']:
            transformed_review.append('not')
        transformed_review.append(tagged_word[0])
    return " ".join(transformed_review)
# Example usage:
for (filename, review, score) in imdb_sentiment_reader(dataset_type='val', sentiment='pos'):
    print "Original review: "
    print review
    print "Transformed review:" 
    transformed = baseline_transform_func(filename, review, score)
    print transformed
    break

Original review: 
I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.
Transformed review:
I went and saw this movie not last night after being coaxed to by a not few friends of mine . I 'll admit that I was not reluctant to see it because from what I knew o

## Baseline Evaluation

In [16]:
train_reader = imdb_sentiment_reader(dataset_type='train', sentiment='both')
test_reader = imdb_sentiment_reader(dataset_type='val', sentiment='both')
default_evaluator = DefaultEvaluator(verbose=True)
baseline_runner = ExperimentRunner(train_reader, test_reader, baseline_transform_func, 
                               evaluator=default_evaluator, verbose=True)
baseline_runner.run_experiment()

Building positive bigram list...
Now on: 0
Now on: 1000
Now on: 2000
Now on: 3000
Now on: 4000
Now on: 5000
Now on: 6000
Now on: 7000
Now on: 8000
Now on: 9000
Now on: 10000
Now on: 11000
Now on: 12000
Building negative bigram list...
Now on: 0
Now on: 1000
Now on: 2000
Now on: 3000
Now on: 4000
Now on: 5000
Now on: 6000
Now on: 7000
Now on: 8000
Now on: 9000
Now on: 10000
Now on: 11000
Now on: 12000
Now evaluating: 0
Current mean score: nan
Finished evaluating 4999 test reviews.
Mean score: 0.0384859418977


In [20]:
print np.median(baseline_runner.scores)

0.356741526233


## Oracle Evaluation

In [101]:
oracle_train_reader = imdb_sentiment_reader(dataset_type='train', sentiment='both')
# Iterate over the 'train', which is a hacky way of just reading in the oracle original documents
oracle_folder = "../Oracle"
oracle_test_reader = imdb_sentiment_reader(dataset_type='train', sentiment='both', folder_location=oracle_folder_location)

all_oracle_filenames = os.listdir(os.path.join(oracle_folder, 'oracle_transformed'))

def oracle_transform_func(filename, review, score):
    """
    Oracle transformation: find the corresponding review in the oracle_transformed directory.
    """
    review_prefix = filename.split(".")[0]
    matching_files = [oracle_file for oracle_file in all_oracle_filenames if oracle_file.startswith(review_prefix)]
    if len(matching_files) != 1:
        raise Exception("More than one matching oracle file: " + str(matching_files))
    else:
        # Return the text of the transformed file
        with open(os.path.join(oracle_folder, 'oracle_transformed', matching_files[0])) as f:
            transformed = f.read()
        return transformed.decode('utf-8')
# Example use:
for index, (filename, review, score) in enumerate(oracle_test_reader):
    print filename
    print index
    print review
    transformed = oracle_transform_func(filename, review, score)
    print transformed
    break

default_evaluator = DefaultEvaluator(verbose=True)
oracle_runner = ExperimentRunner(oracle_train_reader, oracle_test_reader, oracle_transform_func, 
                               evaluator=default_evaluator, verbose=True)
oracle_runner.run_experiment()

10_7.txt
0
If you had asked me how the movie was throughout the film, I would have told you it was great! However, I left the theatre feeling unsatisfied. After thinking a little about it, I believe the problem was the pace of the ending. I feel that the majority of the movie moved kind of slow, and then the ending developed very fast. So, I would say the ending left me disappointed.<br /><br />I thought that the characters were well developed. Costner and Kutcher both portrayed their roles very well. Yes! Ashton Kutcher can act! Also, the different relationships between the characters seemed very real. Furthermore,I thought that the different plot lines were well developed. Overall, it was a good movie and I would recommend seeing it.<br /><br />In conclusion: Good Characters, Great Plot, Poorly Written/Edited Ending. Still, Go See It!!!
If you had asked me how the movie was throughout the film, I would have told you it was terrible! I left the theatre feeling unsatisfied. After think

In [102]:
print np.median(oracle_runner.scores)

0.742032967033
